In [4]:
'''
notebook for playing around with exporting files and georefrencing them in python

eventualy this will be tied together with alot of the scripts from sahi fiftyone into a smooth workflow

'''



# first check the wd
# this should be AIggregation folder. change if needed

import os
os.getcwd()

#os.chdir("..")


In [1]:
# export a test image with its label boxes


import fiftyone as fo

# Import annotated test image into fiftyone dataset type
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = "datasets/test_image/test.yaml"
)


# then export it again
export_dir = "datasets/export_test"
label_field = "ground_truth"  


dataset.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field
    )




 100% |█████████████████████| 1/1 [110.6ms elapsed, 0s remaining, 9.3 samples/s] 
Directory 'datasets/export_test' already exists; export will be merged with existing files
 100% |█████████████████████| 1/1 [33.1ms elapsed, 0s remaining, 30.2 samples/s] 


In [ ]:
# maybe easiest is just reading in the labels file (.csv) from the export and editing it into coordinates

# the name of the file containing the export labels would match the import file names - so ortho something

# multiply csv by image dimensions to get pixel coordinates
# use csv of pixel coordinate points to create a raster 
# use the affine transformation calculated by georefrencng the ortho image to do an identical transformation on the labels raster
# or just give it new meta data 






In [ ]:
# test out rasterio 
# and how to georef image via coordinates


#import required libraries
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show



# open ungeoreferenced raster
unRefRaster = rasterio.open('datasets/drone_ortho/ortho_clip_23april.png')
unRefRaster


# view raster
show(unRefRaster)


# show raster shape 
# this is needed to adjust detection points to pixel coordinates
print("image pixel dimensions are:")
print(unRefRaster.read(1).shape)



# now insert ground control points matching pixels to gps coordinates
# the best way is to open in qgis


# this  currently georeferences back to the orthophoto and the drones GPS

# NE corner of Holford plaque
gcp1 = rasterio.control.GroundControlPoint(col = 10885.28, row=1760.80, x=379256.53562, y=4699478.51867)
gcp1

# NE corner of Brooks plaque
gcp2 = rasterio.control.GroundControlPoint(col = 22572.80, row=7148.15, x=379262.19756, y=4699478.42630)
gcp2

# NE corner of Mabee plaque
gcp3 = rasterio.control.GroundControlPoint(col = 21592.38, row=914.66, x=379260.69707, y=4699480.76276)
gcp3

# NE corner of margorie mabee plaque
gcp4 = rasterio.control.GroundControlPoint(col = 28921.56,row=276.74, x=379263.534649, y=4699482.321738)
gcp4

# stalk of pinecone SW corner of image
gcp5 = rasterio.control.GroundControlPoint(col = 4463.3, row=2763.6, x=379255.06961, y=4699474.29732)
gcp5



# list of selected gcps
gcp = [gcp1, gcp2, gcp3, gcp4, gcp5]
gcp

# get affine transformation
transformation = rasterio.transform.from_gcps(gcp)
print(" ")
print("affine transformation:")
print(transformation)



#define output raster
outputPath = 'datasets/export_test/geo_test/georefRaster.tif'


#create raster and write bands
# by using the transformation and crs 
with rasterio.open(
    outputPath, # specify output raster file path
    'w', # 'w' to specify writing mode
    driver='GTiff', #name of deisred format driver
    height=unRefRaster.read(1).shape[0], #height, in this case taken from unreferenced raster
    width=unRefRaster.read(1).shape[1], #width, in this case taken from unreferenced raster
    count=3, # how many dataset bands (3 for standard color)
    dtype=unRefRaster.read(1).dtype, #data type of dataset
    crs=rasterio.crs.CRS.from_epsg(32618), #CRS.  (WGS84 18N)
    transform=transformation, # specify affine transformation to be used
) as dst: # write out bands
    dst.write(unRefRaster.read(1), 1)
    dst.write(unRefRaster.read(2), 2)
    dst.write(unRefRaster.read(3), 3)

#show georeferenced raster
geoRaster = rasterio.open(outputPath)
show(geoRaster)





In [9]:
# now another way would be just to copy over the metadata from the original orthophoto
# if this works we should be able to copy it over to a raster of bouding boxes aswell


# JK this doesnt work. I think the meta may only save the origin and dimensions of the image but we need a full transformation

import rasterio

# open georef raster and save  meta data

with rasterio.open('datasets/drone_ortho/ortho_clip_23april.tif', 'r') as ref:
    meta = ref.meta


meta

# open unref raster


with rasterio.open('datasets/drone_ortho/ortho_clip_23april.png', 'r') as unref:
    HR = unref.read()




# Save the changed raster
with rasterio.open('datasets/export_test/geo_test/metaswap.tif', 'w', **meta) as dst:
    dst.write(HR)


/home/tmueller/.local/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [ ]:


# open unref raster

unRefRaster = rasterio.open('datasets/drone_ortho/ortho_clip_23april.png')
unRefRaster



# save unref raster with meta data



# Do any transformation you like (on the numpy array)
HR_changed = HR[HR < 0] = 0

# Save the changed raster
with rio.open('C:\\Users\\dataset\\S30W051_TEST.tif', 'w', **meta) as dst:
    dst.write(HR_change)





with rasterio.open(
    outputPath, # specify output raster file path
    'w', # 'w' to specify writing mode
    driver='GTiff', #name of deisred format driver
    height=unRefRaster.read(1).shape[0], #height, in this case taken from unreferenced raster
    width=unRefRaster.read(1).shape[1], #width, in this case taken from unreferenced raster
    count=3, # how many dataset bands (3 for standard color)
    dtype=unRefRaster.read(1).dtype, #data type of dataset
    crs=rasterio.crs.CRS.from_epsg(32618), #CRS.  (WGS84 18N)
    transform=transformation, # specify affine transformation to be used
) as dst: # write out bands
    dst.write(unRefRaster.read(1), 1)
    dst.write(unRefRaster.read(2), 2)
    dst.write(unRefRaster.read(3), 3)



